In [4]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (13.9s)

No dependencies to install or update


# Option valuation model based on historical prices

In [2]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    google_colab = importlib.import_module('google.colab')
    google_colab.drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [6]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: 2024-02-03 08:00:00+00:00 (71 hours left)
current price: 2336.84
-------------------------------
trading put 2300 and call 2350 (hit ratio: 45%)
cost: 0.025 / value: 0.035, benefit/cost = 1.4x
($) cost: 57.25 / value: 81.07, average gain = 23.81
-------------------------------
trading put 2275 and call 2375 (hit ratio: 41%)
cost: 0.017 / value: 0.027, benefit/cost = 1.6x
($) cost: 40.89 / value: 63.75, average gain = 22.85
-------------------------------
trading put 2250 and call 2400 (hit ratio: 37%)
cost: 0.012 / value: 0.021, benefit/cost = 1.7x
($) cost: 28.98 / value: 50.21, average gain = 21.23
-------------------------------
trading put 2225 and call 2425 (hit ratio: 31%)
cost: 0.009 / value: 0.017, benefit/cost = 2.0x
($) cost: 20.10 / value: 39.50, average gain = 19.40


[{'put': 2275.0, 'call': 2375.0, 'quantity': 0},
 {'put': 2250.0, 'call': 2400.0, 'quantity': 1}]

In [5]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2150.0,190.873426,NaN,0.0895,5.157967,0.002208,0.0006
2200.0,144.401912,NaN,0.0630,8.686452,0.003719,0.0012
2250.0,100.562868,NaN,0.0430,14.847408,0.006356,0.0027
2300.0,62.035920,NaN,0.0260,26.320460,0.011268,0.0070
2325.0,46.283084,NaN,0.0170,35.567624,0.015227,0.0110
2350.0,34.259760,0.014667,0.0115,48.544300,NaN,0.0165
2375.0,25.545124,0.010936,0.0080,64.829664,NaN,0.0260
2400.0,19.084275,0.008170,0.0050,83.368815,NaN,0.0330
2450.0,10.723790,0.004591,0.0023,125.008330,NaN,0.0525
